In [27]:
import pandas as pd
import numpy as np

In [28]:
# read file, sheet_name=None to read multiple sheets
dict_df = pd.read_excel("../data/raw/Moving annual rent by suburb - March Quarter 2021.xlsx", sheet_name = None, header = None)
# Source of Hisotrical rental data: https://www.dhhs.vic.gov.au/past-rental-reports
# Rental Report 2021: Moving annualrents by suburb - March

In [43]:
# Define function - gets the year of the last four bits of a string

def get_year(s):
    """
    get year
    :param s: acquire year in the xlsx
    :returns: np.nan
    """
    try:
        return int(s[-4:])  # last four digits
    except Exception:
        return np.nan

In [44]:
def get_month(s):
    """
    get month
    :param s: acquire month in the xlsx
    :returns: np.nan
    """
    try:
        return s[:3]
    except Exception:
        return np.nan

In [45]:
is_first = True
for sheet in dict_df:
    if is_first:
        df = dict_df[sheet].iloc[2:4, :].copy()  # header
        df = df.fillna(method = "pad", axis = 1)  # Fill in the missing values with the previous column
        is_first = False
    df_sheet = dict_df[sheet].iloc[4:, ].copy()
    df_sheet["types"] = sheet  # Adding a 'type' column
    df = df.append(df_sheet)  # merge to the main table
del df[0]

In [46]:
df_T = df.T  # Transpose in numpy, for logical indexing
df_T["year"] = df_T.iloc[:, 0].apply(get_year)  # apply (get_year) to obtain column 'year'
df_T["month"] = df_T.iloc[:, 0].apply(get_month)  # apply (get_month) to obtain column 'month'
df_T = df_T.loc[((df_T["year"] >= 2016) & (df_T["year"] <= 2021)&(df_T["month"] == "Jun")) | df_T["year"].isna()]  # Keep rows with year of 2016-2021 and null values
df_T = df_T.loc[(df_T[3] == "Median") | df_T[3].isna()]  # keep median column
del df_T[3], df_T[2]  # remove unnecessary columns
del df_T["month"]  # remove month column
del df_T["year"]  # remove year column
df_all = df_T.T  # transpose
df_all.columns = ["Sub", 2016, 2017, 2018, 2019, 2020, "Type"]  # Set the column name

In [47]:
is_first = True
for year in range(2016, 2021):
    df_year = df_all[["Sub", "Type"]].copy()
    df_year["Year"] = year
    df_year["Median_Price"] = df_all[year].copy()
    if is_first:
        df_result = df_year.copy()
        is_first = False
    else:
        df_result = df_result.append(df_year)

In [48]:
df_result

,Sub,Type,Year,Median_Price
4,Albert Park-Middle Park-West St Kilda,1 bedroom flat,2016,325
5,Armadale,1 bedroom flat,2016,310
6,Carlton North,1 bedroom flat,2016,330
7,Carlton-Parkville,1 bedroom flat,2016,330
8,CBD-St Kilda Rd,1 bedroom flat,2016,380
...,...,...,...,...
158,Wanagaratta,All properties,2020,300
159,Warragul,All properties,2020,350
160,Warrnambool,All properties,2020,340
161,Wodonga,All properties,2020,330


In [49]:
# output to csv
df_result.to_csv("../data/curated/historical_data.csv", index = False)